# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 250

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 300

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            #tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            #tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### Choose the optimizer and the loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [5]:
NUM_EPOCHS = 10

model.fit(train_data, epochs = NUM_EPOCHS, steps_per_epoch=14, validation_steps=14, validation_data=(validation_inputs, validation_targets), verbose=2)

Train for 14 steps, validate on 6000 samples
Epoch 1/10
14/14 - 3s - loss: 1.3426 - accuracy: 0.6654 - val_loss: 0.6145 - val_accuracy: 0.8105
Epoch 2/10
14/14 - 0s - loss: 0.5032 - accuracy: 0.8489 - val_loss: 0.3820 - val_accuracy: 0.8913
Epoch 3/10
14/14 - 0s - loss: 0.3610 - accuracy: 0.8920 - val_loss: 0.3272 - val_accuracy: 0.9085
Epoch 4/10
14/14 - 0s - loss: 0.3128 - accuracy: 0.9051 - val_loss: 0.3229 - val_accuracy: 0.9097
Epoch 5/10
14/14 - 0s - loss: 0.3167 - accuracy: 0.9069 - val_loss: 0.2833 - val_accuracy: 0.9167
Epoch 6/10
14/14 - 0s - loss: 0.2644 - accuracy: 0.9200 - val_loss: 0.2492 - val_accuracy: 0.9303
Epoch 7/10
14/14 - 0s - loss: 0.2439 - accuracy: 0.9271 - val_loss: 0.2507 - val_accuracy: 0.9280
Epoch 8/10
14/14 - 0s - loss: 0.2563 - accuracy: 0.9260 - val_loss: 0.2125 - val_accuracy: 0.9428
Epoch 9/10
14/14 - 0s - loss: 0.2143 - accuracy: 0.9357 - val_loss: 0.2020 - val_accuracy: 0.9442
Epoch 10/10
14/14 - 0s - loss: 0.1845 - accuracy: 0.9460 - val_loss: 0.19

## Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1875 - accuracy: 0.9432


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.19. Test accuracy: 94.32%
